In [47]:
import requests as r
from dotenv import load_dotenv
load_dotenv()
import os
key = os.getenv("RIOTKEY")
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36",
    "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": key
}

In [15]:
response = r.get("https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/idashhe",headers=headers)
body = response.json()
puuid = body["puuid"]
print(puuid)

OAZVEmbLTrBSLLkjJes4AbQXlua8fT89_gCkNZiRzDDKWj_7IHew4gxMen7gILrQJgFebehnKcU2yA


In [132]:
response = r.get(f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=100&type=ranked",headers=headers)
matches = response.json()

In [134]:
all_participant_data = []
all_timelines = []
for match in matches:
    try:
        response = r.get(f"https://americas.api.riotgames.com/lol/match/v5/matches/{match}",headers=headers)
        match_data = response.json()
        participants = pd.DataFrame(match_data["info"]["participants"])[["participantId","summonerName","individualPosition","teamId"]]
        participants["match"] = match
        all_participant_data.append(participants)
        response = r.get(f"https://americas.api.riotgames.com/lol/match/v5/matches/{match}/timeline",headers=headers)
        timeline = response.json()
        actions = []
        for frame in timeline["info"]["frames"]:
            actions.extend(frame["events"])
        example_timeline = pd.DataFrame(actions)
        example_timeline["match"] = match
        all_timelines.append(example_timeline)
    except Exception as e:
        print(str(e))

'info'
'info'
'info'
'info'
'info'
'info'
'info'
'info'
'info'
'info'
'info'
'info'
'info'
'info'
'info'
'info'
'info'
'info'
'info'
'info'
'info'
'info'
'info'
'info'
'info'
'info'
'info'
'info'
'info'
'info'
'info'
'info'
'info'
'info'
'info'
'info'
'info'
'info'


KeyboardInterrupt: 

In [135]:
match_parts = pd.concat(all_participant_data)
match_time = pd.concat(all_timelines)

In [136]:
match_parts.to_csv("match_participants.csv")
match_time.to_csv("match_time.csv")

In [137]:
kills = match_time[match_time["type"]=="CHAMPION_KILL"]

In [140]:
relevant = []
for col in kills.columns:
    if len([x for x in kills[col].isna() if x]) < 1:
        relevant.append(col)
cleaned = kills[relevant]

In [147]:
match_parts

,participantId,summonerName,individualPosition,teamId,match
0,1,sherben023,TOP,100,NA1_4243988855
1,2,MasterAshe02,JUNGLE,100,NA1_4243988855
2,3,Antifa hooligan,MIDDLE,100,NA1_4243988855
3,4,ISellRooms,BOTTOM,100,NA1_4243988855
4,5,ichiru,UTILITY,100,NA1_4243988855
...,...,...,...,...,...
5,6,bbell3000,TOP,200,NA1_4237608034
6,7,JukeMaister,JUNGLE,200,NA1_4237608034
7,8,Rafik,MIDDLE,200,NA1_4237608034
8,9,Hyme,BOTTOM,200,NA1_4237608034


In [149]:
roles = []
teams = []
ismyteams = []
for row in cleaned.iterrows():
    try:
        killerId = int(row[1]["killerId"])
        match = row[1]["match"]
        da_match = match_parts[(match_parts["match"]==match) & (match_parts["participantId"]==killerId)].iloc[0]
        role = da_match["individualPosition"]
        team = da_match["teamId"]
        my_team = match_parts[(match_parts["match"]==match) & (match_parts["summonerName"]=="idashhe")].iloc[0]["teamId"]
        ismyteam = team == my_team
        roles.append(role)
        teams.append(team)
        ismyteams.append(ismyteam)
    except:
        roles.append("idk")
        teams.append("idk")
        ismyteams.append("idk")
cleaned["killerRole"] = roles
cleaned["killerTeam"] = teams
cleaned["killerismyteam"] = ismyteams
roles = []
teams = []
for row in cleaned.iterrows():
    try:
        killerId = int(row[1]["victimId"])
        match = row[1]["match"]
        da_match = match_parts[(match_parts["match"]==match) & (match_parts["participantId"]==killerId)].iloc[0]
        role = da_match["individualPosition"]
        team = da_match["teamId"]
        roles.append(role)
        teams.append(team)
    except:
        roles.append("idk")
        teams.append("idk")
cleaned["victimRole"] = roles
cleaned["victimTeam"] = teams

C:\Users\eric\AppData\Local\Temp/ipykernel_10328/1436438467.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned["killerRole"] = roles
C:\Users\eric\AppData\Local\Temp/ipykernel_10328/1436438467.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned["killerTeam"] = teams
C:\Users\eric\AppData\Local\Temp/ipykernel_10328/1436438467.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

In [150]:
cleaned.to_csv("kills")

In [301]:
early_game = cleaned[cleaned["timestamp"]<=300000]

In [302]:
roles = early_game["killerRole"].unique()
match_analysises = []
for match in early_game["match"].unique():
    match_data = early_game[early_game["match"]==match]
    for role in roles:
        match_analysis = {}
        my_team_role_kill_count = match_data[(match_data["killerismyteam"]==True) & \
                                             (match_data["killerRole"]==role)].index.size
        enemy_team_role_kill_count = match_data[(match_data["killerismyteam"]==False) & \
                                             (match_data["killerRole"]==role)].index.size
        match_analysis[f"my_team_kc"] = my_team_role_kill_count
        match_analysis[f"enemy_team_kc"] = enemy_team_role_kill_count
        match_analysis["role"] = role
        match_analysis["match"] = match
        match_analysises.append(match_analysis)

In [303]:
manalysis = pd.DataFrame(match_analysises)
manalysis["advantage"] = manalysis["my_team_kc"] >= manalysis["enemy_team_kc"]
manalysis["quant_advantage"] = manalysis["my_team_kc"] - manalysis["enemy_team_kc"]

In [310]:
role = "MIDDLE"
role_manalysis = manalysis[manalysis["role"]==role]
adcount = role_manalysis[role_manalysis["advantage"]==True].index.size
disadcount = role_manalysis[role_manalysis["advantage"]==False].index.size
quantad = role_manalysis["quant_advantage"].mean()
print(adcount,disadcount,quantad)

48 9 0.08771929824561403


In [226]:
highlevel = manalysis.groupby("match").sum().reset_index()
highlevel

,match,my_team_kc,enemy_team_kc,advantage
0,NA1_4237685070,19,12,7
1,NA1_4237835278,18,10,8
2,NA1_4237900871,10,10,0
3,NA1_4237927501,18,12,6
4,NA1_4237934566,6,15,-9
5,NA1_4238042350,23,14,9
6,NA1_4238044206,17,4,13
7,NA1_4238058357,8,9,-1
8,NA1_4238090110,11,16,-5
9,NA1_4238095548,20,9,11


In [213]:
early_game[(early_game["killerismyteam"]==False) & (early_game["killerRole"]=="TOP")].index.size

94

In [207]:
early_game[(early_game["killerismyteam"]==True) & ((early_game["killerRole"]=="BOTTOM") | (early_game["killerRole"]=="UTILITY"))].index.size

353

In [208]:
early_game[(early_game["killerismyteam"]==False) & ((early_game["killerRole"]=="BOTTOM") | (early_game["killerRole"]=="UTILITY"))].index.size

334